# Trabalho 01 – Implementação do Backpropagation em Python (MLP)

Este notebook é para o trabalho de **Implementação do Backpropagation em Python** da disciplina de **Redes Neurais Artificiais (RNA)**.

### Enunciado (resumido)

- Implementar e testar um **Multi Layer Perceptron (MLP)** "na mão" (usando Python / NumPy).
- Comparar o resultado com o **MLP implementado no Scikit-Learn**.
- O MLP implementado deve aceitar parâmetros para:
  - quantidade de camadas;
  - quantidade de neurônios em cada camada;
  - função de ativação utilizada em cada camada.

A ideia aqui é mais didática: eu ainda não tenho muita prática com redes neurais,
então provavelmente dá pra melhorar o código, mas o objetivo principal é ver o 
**backpropagation funcionando** e conseguir comparar com a biblioteca.


## 1. Preparação dos dados

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  # para investigar a curva de erro

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Aqui eu escolhi o make_moons porque é um problema simples 2D, mas não linear.
# Fica mais fácil de testar se o MLP aprendeu alguma coisa "de verdade".
dados_X, rotulos_y = make_moons(n_samples=1000, noise=0.2, random_state=42)

# Separando em treino e teste (80% / 20%)
X_treino, X_teste, y_treino, y_teste = train_test_split(
    dados_X, rotulos_y, test_size=0.2, random_state=42
)

# Normalização: nas aulas e na documentação falam que ajuda bastante em MLP,
# então resolvi padronizar as features.
normalizador = StandardScaler()
X_treino = normalizador.fit_transform(X_treino)
X_teste = normalizador.transform(X_teste)

qtd_caracteristicas = X_treino.shape[1]
qtd_classes = len(np.unique(y_treino))

print("qtd_caracteristicas:", qtd_caracteristicas)
print("qtd_classes:", qtd_classes)

## 2. Implementação do MLP "do zero" (NumPy)

Aqui eu implementei uma classe `MLP` bem simples, com:

- `tamanhos_camadas`: lista com o tamanho de cada camada (entrada, ocultas e saída)
- `ativacoes`: lista com a função de ativação de cada camada (oculta/saída)
- parâmetros de treinamento:
  - `taxa_aprendizado`
  - `epocas`
  - `semente` (para reprodução dos resultados)

A parte em que eu mais tive dúvida foi o **backpropagation**, principalmente:
- conferir se as dimensões das matrizes estavam certas (W, delta, etc.)
- ter certeza de que eu estava usando a derivada da ativação correta em cada camada.

Também acrescentei um `historico_erro` para conseguir plotar a curva de treinamento
e ver melhor quando a rede começa de fato a aprender.


In [ ]:
class MLP:
    def __init__(
        self,
        tamanhos_camadas,
        ativacoes,
        taxa_aprendizado=0.1,
        epocas=1000,
        semente=None,
        mostrar_erro=True,
    ):
        """
        tamanhos_camadas: lista com o tamanho de cada camada, incluindo entrada e saída.
            Ex: [2, 10, 5, 2]  -> entrada com 2, duas camadas ocultas (10 e 5) e saída com 2 neurônios.
        ativacoes: lista com o nome da ativação pra cada camada oculta/saída.
            Ex: ["tanh", "tanh", "softmax"]  (mesmo tamanho de tamanhos_camadas - 1)
        """
        if len(ativacoes) != len(tamanhos_camadas) - 1:
            raise ValueError("A lista de ativacoes deve ter len(tamanhos_camadas) - 1")


        self.tamanhos_camadas = tamanhos_camadas
        self.ativacoes = ativacoes
        self.taxa_aprendizado = taxa_aprendizado
        self.epocas = epocas
        self.semente = semente
        self.mostrar_erro = mostrar_erro

        # número de camadas que têm pesos (não conta a entrada)
        self.num_camadas = len(tamanhos_camadas) - 1

        # vou guardar o histórico de erro pra conseguir investigar depois
        self.historico_erro = []

        # só por precaução, inicializo rotulos_ como None
        self.rotulos_ = None

        self._inicializar_pesos()

    def _inicializar_pesos(self):
        # aqui eu mudei a inicialização porque com pesos muito pequenos
        # a rede parecia ficar "travada" em algo quase linear
        gerador = np.random.RandomState(self.semente)
        self.pesos = []
        self.biases = []

        for i in range(self.num_camadas):
            entradas = self.tamanhos_camadas[i]
            saidas = self.tamanhos_camadas[i + 1]

            # tentativa de inicialização um pouco mais "esperta" (tipo Xavier simples)
            # (pra ser sincero eu não lembro o nome certinho da técnica, vi algo parecido em material de rede)
            limite = np.sqrt(1.0 / entradas)
            w = gerador.uniform(-limite, limite, size=(entradas, saidas))
            b = np.zeros(saidas)

            self.pesos.append(w)
            self.biases.append(b)

    def _funcao_ativacao(self, z, nome):
        # aqui resolvi usar nomes em inglês mesmo, tipo 'tanh', 'relu', 'sigmoid', 'softmax'
        if nome == "sigmoid":
            return 1.0 / (1.0 + np.exp(-z))
        elif nome == "tanh":
            return np.tanh(z)
        elif nome == "relu":
            return np.maximum(0, z)
        elif nome == "linear":
            return z
        elif nome == "softmax":
            # normalmente uso softmax só na última camada (saída)
            exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
            return exp_z / np.sum(exp_z, axis=1, keepdims=True)
        else:
            raise ValueError(f"Função de ativação desconhecida: {nome}")

    def _derivada_ativacao(self, z, nome):
        # derivadas básicas das ativações
        if nome == "sigmoid":
            sig = 1.0 / (1.0 + np.exp(-z))
            return sig * (1.0 - sig)
        elif nome == "tanh":
            t = np.tanh(z)
            return 1.0 - t**2
        elif nome == "relu":
            return (z > 0).astype(float)
        elif nome == "linear":
            return np.ones_like(z)
        elif nome == "softmax":
            # pra softmax + cross-entropy eu não uso essa derivada aqui
            return np.ones_like(z)
        else:
            raise ValueError(f"Função de ativação desconhecida: {nome}")

    def _forward(self, X):
        """
        Passada pra frente (forward):
        - calcula z^l = a^{l-1} W^l + b^l
        - aplica a função de ativação pra obter a^l

        Aqui preciso guardar:
        - todas as ativações (a^l)
        - todos os z^l

        porque o backpropagation depois usa isso.
        """
        ativacao = X
        ativacoes = [ativacao]  # a^0 = X (entrada)
        zs = []                 # lista com todos os z^l

        for i in range(self.num_camadas):
            w = self.pesos[i]
            b = self.biases[i]

            z = ativacao @ w + b  # produto matricial + bias
            zs.append(z)

            nome_ativ = self.ativacoes[i]
            ativacao = self._funcao_ativacao(z, nome_ativ)
            ativacoes.append(ativacao)

        return zs, ativacoes

    def _calcular_erro(self, y_one_hot, y_pred):
        """
        Se a última camada for softmax, uso cross-entropy.
        Se não, faço um MSE simples (deixei aqui mais pra generalizar).
        """
        ultima_ativ = self.ativacoes[-1]
        if ultima_ativ == "softmax":
            eps = 1e-8
            return -np.mean(np.sum(y_one_hot * np.log(y_pred + eps), axis=1))
        else:
            return 0.5 * np.mean(np.sum((y_pred - y_one_hot) ** 2, axis=1))

    def _backpropagation(self, zs, ativacoes, y_one_hot):
        """
        Implementação do backpropagation.

        Aqui foi a parte que eu mais tive dificuldade.
        A ideia geral:
        - calcular o erro na saída (delta da última camada)
        - ir "voltando" camada por camada, aplicando a regra da cadeia

        zs: lista com os z^l
        ativacoes: lista com os a^l
        y_one_hot: rótulos em formato one-hot
        """
        m = y_one_hot.shape[0]  # quantidade de exemplos

        grad_pesos = [None] * self.num_camadas
        grad_biases = [None] * self.num_camadas

        y_pred = ativacoes[-1]
        ultima_ativ = self.ativacoes[-1]

        # erro na camada de saída
        if ultima_ativ == "softmax":
            # derivada de cross-entropy + softmax fica bem simples: y_pred - y_true
            delta = y_pred - y_one_hot
        else:
            # se fosse MSE, seria algo assim:
            delta = (y_pred - y_one_hot) * self._derivada_ativacao(zs[-1], ultima_ativ)

        # loop de trás pra frente (camadas L, L-1, ..., 1)
        for camada in reversed(range(self.num_camadas)):
            a_anterior = ativacoes[camada]  # a^{l}

            # gradiente em relação aos pesos e biases
            grad_pesos[camada] = a_anterior.T @ delta / m
            grad_biases[camada] = np.mean(delta, axis=0)

            if camada != 0:
                nome_ativ_anterior = self.ativacoes[camada - 1]
                # propagando o erro pra camada anterior
                delta = (delta @ self.pesos[camada].T) * self._derivada_ativacao(
                    zs[camada - 1], nome_ativ_anterior
                )

        return grad_pesos, grad_biases

    def treinar(self, X, y):
        """
        Treinamento usando gradiente descendente simples.

        y pode vir como vetor de inteiros (0,1,2,...) que eu converto pra one-hot.
        """
        X = np.asarray(X)
        y = np.asarray(y)

        # reseta o histórico de erro sempre que treinar de novo
        self.historico_erro = []

        # converte para one-hot se for apenas um vetor de inteiros
        if y.ndim == 1:
            self.rotulos_, indices = np.unique(y, return_inverse=True)
            qtd_classes = len(self.rotulos_)
            y_one_hot = np.eye(qtd_classes)[indices]
        else:
            # se já vier em one-hot, assumo que está ok
            y_one_hot = y
            self.rotulos_ = np.arange(y.shape[1])

        for epoca in range(self.epocas):
            zs, ativacoes = self._forward(X)
            y_pred = ativacoes[-1]

            erro = self._calcular_erro(y_one_hot, y_pred)
            grad_pesos, grad_biases = self._backpropagation(zs, ativacoes, y_one_hot)

            # guarda o erro pra investigar depois
            self.historico_erro.append(erro)

            # atualização dos pesos (gradiente descendente)
            for i in range(self.num_camadas):
                self.pesos[i] -= self.taxa_aprendizado * grad_pesos[i]
                self.biases[i] -= self.taxa_aprendizado * grad_biases[i]

            if self.mostrar_erro and (epoca % 100 == 0 or epoca == self.epocas - 1):
                print(f"Época {epoca}, erro = {erro:.4f}")

        return self

    def prever_proba(self, X):
        X = np.asarray(X)
        _, ativacoes = self._forward(X)
        return ativacoes[-1]

    def prever(self, X):
        probas = self.prever_proba(X)
        indices = np.argmax(probas, axis=1)
        # se por algum motivo rotulos_ não existir (por ex, se eu esquecer de chamar treinar),
        # eu devolvo os índices brutos mesmo (0, 1, 2, ...)
        if self.rotulos_ is None:
            return indices
        return self.rotulos_[indices]

## 3. Treinamento do MLP implementado

In [ ]:
# Arquitetura do MLP manual:
# entrada -> 20 neurônios -> 20 neurônios -> saída (qtd_classes)
tamanhos_camadas = [qtd_caracteristicas, 20, 20, qtd_classes]

# Ativações correspondentes às camadas (sem contar a entrada)
ativacoes = ["tanh", "tanh", "softmax"]

mlp_manual = MLP(
    tamanhos_camadas=tamanhos_camadas,
    ativacoes=ativacoes,
    taxa_aprendizado=0.05,  # taxa de aprendizado um pouco menor
    epocas=3000,            # mais épocas pra ver se a loss desce bem
    semente=42,
    mostrar_erro=True,
)

mlp_manual.treinar(X_treino, y_treino)

y_pred_mlp_manual = mlp_manual.prever(X_teste)
acuracia_manual = accuracy_score(y_teste, y_pred_mlp_manual)

print("Acurácia MLP implementado na mão:", acuracia_manual)

## 4. Investigando a curva de erro (loss)

Aqui eu quis ver melhor como o erro se comporta ao longo das épocas.
Usei o `historico_erro` que foi guardado durante o treinamento e plotei
a curva abaixo.

Isso ajuda a enxergar melhor em que momento o modelo começa a sair do
“chute aleatório” (erro ≈ 0,693 em classificação binária com softmax).

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(mlp_manual.historico_erro)
plt.xlabel("Época")
plt.ylabel("Erro (loss)")
plt.title("Curva de treinamento do MLP (implementado na mão)")
plt.grid(True)
plt.show()

## 5. Visualizando a fronteira de decisão do MLP implementado

Além da curva de erro, eu quis ver também a **fronteira de decisão** aprendida
pelo MLP que eu implementei. 

Esse tipo de gráfico ajuda a enxergar se o modelo realmente separou bem as duas
classes no plano 2D (no caso do `make_moons`).

In [ ]:
def plotar_fronteira_decisao_mlp_manual(modelo, X, y, titulo="Fronteira de decisão - MLP manual"):
    """
    Desenha a fronteira de decisão em 2D para o MLP implementado na mão.

    Obs: aqui eu uso X já normalizado (X_treino), porque foi assim que eu treinei o modelo.
    """
    passo = 0.02  # quanto menor o passo, mais detalhado (e mais pesado)

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, passo),
        np.arange(y_min, y_max, passo)
    )

    X_grade = np.c_[xx.ravel(), yy.ravel()]

    Z = modelo.prever(X_grade)
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(6, 4))
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors="k", alpha=0.7)
    plt.title(titulo)
    plt.xlabel("feature 1 (normalizada)")
    plt.ylabel("feature 2 (normalizada)")
    plt.show()


# aqui eu chamo a função para o meu MLP manual, usando os dados de treino
plotar_fronteira_decisao_mlp_manual(mlp_manual, X_treino, y_treino)

## 6. Comparação com o MLPClassifier do Scikit-Learn

In [ ]:
# Agora eu uso o MLPClassifier do Scikit-Learn com uma arquitetura parecida:
mlp_sklearn = MLPClassifier(
    hidden_layer_sizes=(20, 20),
    activation="tanh",
    solver="sgd",          # pra ficar mais parecido com gradiente descendente simples
    learning_rate_init=0.05,
    max_iter=3000,
    random_state=42,
)

mlp_sklearn.fit(X_treino, y_treino)

y_pred_sklearn = mlp_sklearn.predict(X_teste)
acuracia_sklearn = accuracy_score(y_teste, y_pred_sklearn)

print("Acurácia MLPClassifier (Scikit-Learn):", acuracia_sklearn)

## 7. Visualizando a fronteira de decisão do MLPClassifier (Scikit-Learn)

Para comparar melhor, também plotei a fronteira de decisão do `MLPClassifier`
do Scikit-Learn, usando a mesma ideia de grade de pontos 2D.

Daria para generalizar e fazer uma função única, mas eu preferi separar em duas
funções para não complicar muito a lógica.

In [ ]:
def plotar_fronteira_decisao_sklearn(modelo, X, y, titulo="Fronteira de decisão - MLP (Scikit-Learn)"):
    """
    Desenha a fronteira de decisão em 2D para o MLPClassifier do Scikit-Learn.

    Aqui também uso X normalizado (X_treino), igual no treinamento.
    """
    passo = 0.02

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, passo),
        np.arange(y_min, y_max, passo)
    )

    X_grade = np.c_[xx.ravel(), yy.ravel()]

    Z = modelo.predict(X_grade)
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(6, 4))
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors="k", alpha=0.7)
    plt.title(titulo)
    plt.xlabel("feature 1 (normalizada)")
    plt.ylabel("feature 2 (normalizada)")
    plt.show()


# chamada para o MLP do Scikit-Learn
plotar_fronteira_decisao_sklearn(mlp_sklearn, X_treino, y_treino)

## 8. Conclusão

Neste trabalho eu:

1. Implementei um **MLP do zero** em Python, usando apenas NumPy:
   - a classe `MLP` recebe como parâmetros:
     - `tamanhos_camadas` (quantidade de camadas e neurônios em cada uma);
     - `ativacoes` (função de ativação para cada camada oculta e saída);
     - hiperparâmetros de treinamento (taxa de aprendizado, épocas, etc.).
   - implementei o **forward** e o **backpropagation** manualmente.
   - armazenei o `historico_erro` para investigar a curva de treinamento.

2. Treinei esse MLP em um conjunto de dados de exemplo (`make_moons`), fazendo:
   - separação em treino e teste;
   - normalização das entradas;
   - cálculo da acurácia no conjunto de teste;
   - análise visual da curva de erro ao longo das épocas;
   - visualização da fronteira de decisão aprendida pelo modelo.

3. Treinei também um **MLPClassifier do Scikit-Learn** com uma arquitetura parecida e:
   - comparei a acurácia do meu MLP com a acurácia do MLP do Scikit-Learn;
   - plotei a fronteira de decisão de ambos os modelos.

Os resultados não ficaram exatamente iguais, mas as acurácias ficaram na mesma ordem
de grandeza, o que indica que a implementação do backpropagation está funcionando
de forma razoável.

Ainda tenho dúvidas em alguns detalhes, principalmente na parte de:
- escolha dos hiperparâmetros (taxa de aprendizado, número de épocas);
- inicialização de pesos mais "esperta" (essa que usei foi inspirada em exemplos, mas ainda estou entendendo melhor).

Mesmo assim, consegui cumprir os requisitos principais da tarefa:
- implementar o MLP com parâmetros configuráveis de camadas/neurônios/ativações;
- comparar o desempenho com o MLP do Scikit-Learn;
- e observar, tanto numericamente quanto visualmente, o comportamento do treinamento.


## Referência aos notebooks originais do professor

Durante o desenvolvimento deste trabalho, eu tentei utilizar como base o seguinte notebook,
que em princípio seria a referência principal para backpropagation com uma camada oculta:

- `aula4a_Single_Hidden_Layer_Backpropagation.ipynb`  
  Link informado: https://github.com/fboldt/aulasann/blob/main/aula4a_Single_Hidden_Layer_Backpropagation.ipynb  

No momento da realização do trabalho, esse arquivo **não estava disponível** (não consegui
acessar o conteúdo do notebook).

Como material de apoio conceitual, acabei utilizando principalmente:

- `aula05b_single_hidden_layer.ipynb`  
  Disponível em: https://github.com/fboldt/aulasann/blob/main/aula05b_single_hidden_layer.ipynb  

Esse notebook foi usado para revisar a ideia geral do algoritmo de backpropagation em uma
rede com uma camada escondida, e a partir dele escrevi minha própria classe `MLP` mais genérica.

A comparação com o `MLPClassifier` do Scikit-Learn foi feita com um exemplo implementado por mim,
seguindo o que vimos em aula sobre usar o MLP do Scikit-Learn como "padrão de comparação".

Para contextualizar a organização do repositório do professor, outros arquivos como
`aula05c_multilayer.ipynb` e `aula06a_mlp_scikit_learn.ipynb` parecem dar sequência ao mesmo
tema (multicamadas e uso direto do MLP do Scikit-Learn), mas a implementação final apresentada
aqui foi escrita por mim a partir dos conceitos vistos em aula.
